# 📚 AkademskaKniga.mk - Book Dataset Cleaning & Preprocessing

In [88]:
import pandas as pd
import numpy as np

In [89]:
df= pd.read_csv('../data/original_datasets/akademskakniga_books.csv', encoding='utf-8-sig') 

In [90]:
df.head()

,id,title,author,real_price,sale_price,sale,category,retrieved_at
0,1,Accounting,"Warren, Carl S.",6199.0,4959 Мкд.,1,Accounting & Finance,2025-05-15
1,2,Pricing in General I,"Parodi, Pietro",5399.0,4319 Мкд.,1,Accounting & Finance,2025-05-15
2,3,Behavioral Finance i,"Kubinska, Elzbieta",9399.0,7519 Мкд.,1,Accounting & Finance,2025-05-15
3,4,Handbook of Alternat,-,10099.0,8079 Мкд.,1,Accounting & Finance,2025-05-15
4,5,Using QuickBooks (R),"Barbara), Glenn Owen",4999.0,3999 Мкд.,1,Accounting & Finance,2025-05-15


In [91]:
df = df.drop('id', axis=1)

### 1. Converting the 'Sale' Column to Boolean & rename it

In [92]:
df["sale"] = df["sale"].astype(bool)

In [93]:
df = df.rename(columns={"sale": "IsOnSale"})

### 2. Convert the 'Retrieved At' column to datetime format.

In [94]:
df['retrieved_at'] = pd.to_datetime(df['retrieved_at'])

### 3. Prices
- Before converting to numeric,remove "Мкд.", remove commas and strip whitespace


In [95]:
df['sale_price'] = df['sale_price'].str.replace("Мкд.", "", regex=False).str.replace(",", "").str.strip()

df["real_price"] = pd.to_numeric(df["real_price"], errors="coerce")
df["sale_price"] = pd.to_numeric(df["sale_price"], errors="coerce")

df['sale_price'] = df['sale_price'].astype(float)
df['real_price'] = df['real_price'].astype(float)

### 4. Remove duplicates

In [96]:
df = df.dropna(subset=['real_price'])

In [97]:
duplicates = df[df.duplicated(keep=False)]
duplicates = duplicates.sort_values(by=['title', 'author'])
print("Length of duplicates: ", len(duplicates))

Length of duplicates:  5122


In [98]:
print("Original:", len(df))
df = df.drop_duplicates()
print("After drop_duplicates:", len(df))

Original: 24536
After drop_duplicates: 21393


### 4. Define a function to normalize author names  
- Inconsistent Formats:
    - Some names are written in the `"LastName, FirstName"` format.
    - Others are written in `"FirstName LastName"` or even as a comma-separated list of multiple authors.

- Normalizing names into a consistent format like `"FirstName LastName"` or `"Author1 Author2 Author3"`.

In [99]:
def normalize_author(author):
    if pd.isna(author):
        return author  
    if ',' in author:
        parts = [part.strip() for part in author.split(',')]
        return ' '.join(parts[::-1])
    return author

In [100]:
df['author'] = df['author'].apply(normalize_author)

In [101]:
df.head()

,title,author,real_price,sale_price,IsOnSale,category,retrieved_at
0,Accounting,Carl S. Warren,6199.0,4959.0,True,Accounting & Finance,2025-05-15
1,Pricing in General I,Pietro Parodi,5399.0,4319.0,True,Accounting & Finance,2025-05-15
2,Behavioral Finance i,Elzbieta Kubinska,9399.0,7519.0,True,Accounting & Finance,2025-05-15
3,Handbook of Alternat,-,10099.0,8079.0,True,Accounting & Finance,2025-05-15
4,Using QuickBooks (R),Glenn Owen Barbara),4999.0,3999.0,True,Accounting & Finance,2025-05-15


### 5. Replace '?' in author with null

In [102]:
df.loc[df['author'].str.match(r'^[\?=]+$', na=False), 'author'] = np.nan

### 6. Replace '-' in author with null

In [103]:
df.loc[df['author'] == '-', 'author'] = np.nan

### 7. Replace "-" with spaces only in the 'Author' column, e.g. "Maria Herbert-Liew" → "Maria Herbert Liew"

In [104]:
df['author'] = df['author'].str.replace('-', ' ', regex=False)

### 8. Remove numbers from the 'Author' column,e.g. '1854 19 Oscar Wilde'

In [105]:
df['author'] = df['author'].str.replace(r'\d+', '', regex=True).str.strip().str.replace(r'\s+', ' ', regex=True)

### 9. Remove all parentheses 
- Remove all parentheses  and their content, matched or unmatched, including the parentheses themselves.
- This simplifies your data, especially if the content inside parentheses is often additional info like (editor), (illustrator), or incomplete fragments like (author,  or author).

In [106]:
mask = df['author'].str.contains(r'[\(\)]', regex=True, na=False)

print("Before cleaning:")
df.loc[mask, ['author']]

Before cleaning:


,author
4,Glenn Owen Barbara)
45,Financial (author)
88,Parveen Za (editor)
149,Royal Horticu work)
151,Melan (illustrator)
...,...
25118,Syuhe (illustrator)
25151,GODSS (illustrator)
25155,Tracy Yard (artist)
25156,Tomo (illustrator)


In [107]:
df.loc[mask, 'author'] = df.loc[mask, 'author'].str.replace(r'\([^)]*\)', '', regex=True)
df.loc[mask, 'author'] = df.loc[mask, 'author'].str.replace(r'[\(\)]', '', regex=True)
df.loc[mask, 'author'] = df.loc[mask, 'author'].str.strip()

print("\nAfter cleaning:")
df.loc[mask, ['author']]


After cleaning:


,author
4,Glenn Owen Barbara
45,Financial
88,Parveen Za
149,Royal Horticu work
151,Melan
...,...
25118,Syuhe
25151,GODSS
25155,Tracy Yard
25156,Tomo


### 10. Create Discount Percentage Column(%):
- If IsOnSale is True, calculate discount info(discount percentage)

In [108]:
df['Discount (%)'] = 0.0

mask = (df['IsOnSale'] == True) & (df['real_price'] > 0)
df.loc[mask, 'Discount (%)'] = (
    ((df.loc[mask, 'real_price'] - df.loc[mask, 'sale_price']) / df.loc[mask, 'real_price']) * 100
).round().astype(int)


In [109]:
df.head()

,title,author,real_price,sale_price,IsOnSale,category,retrieved_at,Discount (%)
0,Accounting,Carl S. Warren,6199.0,4959.0,True,Accounting & Finance,2025-05-15,20.0
1,Pricing in General I,Pietro Parodi,5399.0,4319.0,True,Accounting & Finance,2025-05-15,20.0
2,Behavioral Finance i,Elzbieta Kubinska,9399.0,7519.0,True,Accounting & Finance,2025-05-15,20.0
3,Handbook of Alternat,NaN,10099.0,8079.0,True,Accounting & Finance,2025-05-15,20.0
4,Using QuickBooks (R),Glenn Owen Barbara,4999.0,3999.0,True,Accounting & Finance,2025-05-15,20.0


### 11. Missing values

In [110]:
missing_percent = df.isnull().sum() / len(df) * 100
missing_percent = missing_percent.round(2)
missing_percent = missing_percent.reset_index().rename(columns={"index": "column", 0: "percent missing"})
missing_percent

,column,percent missing
0,title,0.00
1,author,1.22
2,real_price,0.00
3,sale_price,0.00
4,IsOnSale,0.00
5,category,0.00
6,retrieved_at,0.00
7,Discount (%),0.00


In [111]:
total_rows = len(df)
missing_authors = df['author'].isna().sum()
non_missing_authors = total_rows - missing_authors

print(f"\n'Author' column:")
print(f"Non-missing values: {non_missing_authors}")
print(f"Missing values: {missing_authors}")
print(f"Percentage missing: {round((missing_authors / total_rows) * 100, 2)}%")



'Author' column:
Non-missing values: 21132
Missing values: 261
Percentage missing: 1.22%


In [112]:
df.to_csv("../data/preprocessed_datasets/akademska_books.csv", encoding='utf-8-sig', index=False)

### Preprocessing Output Summary

Number of Books Scraped

In [113]:
df.shape[0]

21393

Number of categories

In [114]:
total_categories = df['category'].nunique()
print(total_categories)

37


Number of Discounted Books

In [115]:
num_distinct_on_sale = df[df['IsOnSale'] == True]['title'].nunique()
print(num_distinct_on_sale)

19228
